In [14]:
import tensorflow as tf

x = tf.constant(10, dtype=tf.float32)
y = tf.constant(20, dtype=tf.float32)
z = tf.Variable(tf.add(x, y))

# No need for global_variables_initializer() in TensorFlow 2.x

# TensorFlow 2.x automatically initializes variables when they are used in sessions or functions

# Instead of session.run(z), you can simply access the value of z using z.numpy()
print(z.numpy())


30.0
